# Hyperparameter Tuning

In [ ]:
import numpy as np
from data_preperation.dataset import CityDataset
import os

from config import PATH, CITIES, MIN_LABELS, PATCH_SIZE, TEST_CITY

In [ ]:
dataset = CityDataset(
    PATH,
    patch_size=PATCH_SIZE,
    data_name="openEO.tif",
    labels_name="building_mask_dense.tif",
    image_bands=[1, 2, 3, 4, 5, 6],
    min_labels=MIN_LABELS,
    cities=CITIES,
    train=True,
)

dataset_test = CityDataset(
    PATH,
    data_name="openEO.tif",
    labels_name="building_mask_dense.tif",
    image_bands=[1, 2, 3, 4, 5, 6],
    cities=TEST_CITY,
    train=False,
)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# from torch.utils.tensorboard import SummaryWriter
import lightning as L
from typing import Any
from torch.utils.data import DataLoader, Dataset
from lightning import seed_everything
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from models.lightning_utils import LitModule
from models.baseconvnet import ConvNetSimple

from lightning.pytorch.tuner.tuning import Tuner

# model
# convmodel = LitModule(ConvNetSimple())


# trainer
def get_trainer(dirname):
    trainer = L.Trainer(
        default_root_dir=f"model_experiments/hyperparam_tuning/{dirname}",
        callbacks=[
            EarlyStopping(
                monitor="val_loss",
                mode="min",
                patience=2,
            ),
            ModelCheckpoint(
                dirpath=f"model_experiments/hyperparam_tuning/{dirname}",
                monitor="val_loss",
                mode="min",
                save_top_k=1,
                filename="best_model",
            ),
        ],
        # val_check_interval=1,
        fast_dev_run=False,
        num_sanity_val_steps=2,
        max_epochs=10,
        log_every_n_steps=20,
    )
    return trainer

In [ ]:
from itertools import product
import random

b = {"B04": 1, "B03": 2, "B02": 3, "B08": 4, "B12": 5, "B11": 6}
channels = list(b.values())
print(channels)
optimizers = ["sgd", "adamW", "adam"]

grid = []
i = 0
while i < 5:
    num_channels = random.randint(3, 6)
    selected_channels = random.sample(channels, num_channels)
    selected_channels.sort()
    if selected_channels not in grid:
        i += 1
        grid.append(selected_channels)
    else:
        continue
print(grid)

In [ ]:
model_stats = {}
torch.set_float32_matmul_precision("high")  # for tensor cores
test_dl = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=20)
for chs in grid:
    print(chs)
    # load the data
    dataset = CityDataset(
        PATH,
        patch_size=32,
        data_name="openEO.tif",
        labels_name="building_mask_dense.tif",
        image_bands=chs,
        min_labels=0.1,
        cities=CITIES,
        train=True,
    )
    train_ds, val_ds = dataset.train_val_split(val_size=0.1, show_summary=False)
    train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=20)
    val_dl = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=20)

    dataset_test = CityDataset(
        PATH,
        data_name="openEO.tif",
        labels_name="building_mask_dense.tif",
        image_bands=chs,
        cities=TEST_CITY,
        train=False,
    )
    test_dl = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=20)

    for optim in optimizers:
        print(chs, optim)
        # create the model
        model = LitModule(ConvNetSimple(len(chs)), learning_rate=0.001, optimizer=optim)
        # create the trainer
        trainer = get_trainer(
            f'convSimple/{str(chs).replace(", ", "_").replace("[", "_").replace("]", "_")}{optim}'
        )
        # tuner lr
        seed_everything(49)
        tuner = Tuner(trainer=trainer)
        tuner.lr_find(
            model, train_dl, val_dl, min_lr=1e-5, max_lr=0.03, num_training=5000
        )

        # train the model
        trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)
        model_stats[f"{str(chs)}_{optim}"] = dict(
            best_model=trainer.checkpoint_callback.best_model_path,
        )
        # test the model
        best_model = LitModule.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )
        trainer.test(model=model, dataloaders=test_dl)

# UNET Hyperparam Search

In [ ]:
model_stats = {}
from models.unet import UNet

torch.set_float32_matmul_precision("high")  # for tensor cores
test_dl = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=20)
for chs in grid:
    print(chs)
    # load the data
    dataset = CityDataset(
        PATH,
        patch_size=32,
        data_name="openEO.tif",
        labels_name="building_mask_dense.tif",
        image_bands=chs,
        min_labels=0.1,
        cities=CITIES,
        train=True,
    )
    train_ds, val_ds = dataset.train_val_split(val_size=0.1, show_summary=False)
    train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=20)
    val_dl = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=20)

    dataset_test = CityDataset(
        PATH,
        data_name="openEO.tif",
        labels_name="building_mask_dense.tif",
        image_bands=chs,
        cities=TEST_CITY,
        train=False,
    )
    test_dl = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=20)

    for optim in optimizers:
        print(chs, optim)
        # create the model

        # model
        unet = UNet(
            n_channels=len(dataset.get_image_bands()), n_classes=1, bilinear=True
        )
        # UNet implementation uses the BCEWithLogitsLoss, lr of 1e-5 default
        model = LitModule(unet, learning_rate=1e-4, loss=nn.BCEWithLogitsLoss())

        # create the trainer
        trainer = get_trainer(
            f'unet/{str(chs).replace(", ", "_").replace("[", "_").replace("]", "_")}{optim}'
        )

        # tuner lr
        seed_everything(49)
        tuner = Tuner(trainer=trainer)
        tuner.lr_find(
            model, train_dl, val_dl, min_lr=1e-5, max_lr=0.03, num_training=5000
        )

        # train the model
        trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)
        model_stats[f"{str(chs)}_{optim}"] = dict(
            best_model=trainer.checkpoint_callback.best_model_path,
        )
        # test the model
        best_model = LitModule.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )
        trainer.test(model=model, dataloaders=test_dl)